# DEMO nærings API

Denne demoen er ment for å vise hvordan flyten for et sluttbrukersystem kan hente et utkast, gjøre endringer, validere/kontrollere det mot Skatteetatens apier, for å sende det inn via Altinn3

In [ ]:
# eksterne avhengigheter
!pip install python-jose[cryptography] cryptography
!pip install xmltodict

In [1]:
from altinn3 import *
from hent import main_relay, base64_decode_response, decode_dokument
import requests
import base64
import xmltodict
import xml.dom.minidom

## Generer ID-porten token
Tokenet er gyldig i 300 sekunder, rekjørt denne biten om du ikke har kommet frem til Altinn3 biten før 300 sekunder 

In [2]:
idporten_header = main_relay()

https://oidc-ver2.difi.no/idporten-oidc-provider/authorize?scope=openid&acr_values=Level3&client_id=38e634d9-5682-44ae-9b60-db636efe3156&redirect_uri=http%3A%2F%2Flocalhost%3A12345%2Ftoken&response_type=code&state=3Q1S4dmpgJmsDIX9rz3jnw&nonce=1603265479155602&resource=https%3A%2F%2Fmp-test.sits.no%2Fapi%2Feksterntapi%2Fformueinntekt%2Fskattemelding%2F&code_challenge=wmluC7C0i7iOLcjxv5vF_lQqxKBQjQ1Wb7blpl4QiUU=&code_challenge_method=S256&ui_locales=nb


127.0.0.1 - - [21/Oct/2020 09:31:38] "GET /token?code=34ijyTerpgGtQXfBkb10HeQC9eIoiYh6dLMw-AK4JPM&state=3Q1S4dmpgJmsDIX9rz3jnw HTTP/1.1" 200 -


Authorization token received
The token is good, expires in 300 seconds

Bearer eyJraWQiOiJjWmswME1rbTVIQzRnN3Z0NmNwUDVGSFpMS0pzdzhmQkFJdUZiUzRSVEQ0IiwiYWxnIjoiUlMyNTYifQ.eyJzdWIiOiJmdnBRZ1d5eFRfeHJZVVR0RFJpS1gySlhHYjVlT2lodG91Skpoa0J0Nnk0PSIsImlzcyI6Imh0dHBzOlwvXC9vaWRjLXZlcjIuZGlmaS5ub1wvaWRwb3J0ZW4tb2lkYy1wcm92aWRlclwvIiwiY2xpZW50X2FtciI6Im5vbmUiLCJwaWQiOiIwNzExNjAwMDczOCIsInRva2VuX3R5cGUiOiJCZWFyZXIiLCJjbGllbnRfaWQiOiIzOGU2MzRkOS01NjgyLTQ0YWUtOWI2MC1kYjYzNmVmZTMxNTYiLCJhdWQiOiJodHRwczpcL1wvbXAtdGVzdC5zaXRzLm5vXC9hcGlcL2Vrc3Rlcm50YXBpXC9mb3JtdWVpbm50ZWt0XC9za2F0dGVtZWxkaW5nXC8iLCJhY3IiOiJMZXZlbDMiLCJzY29wZSI6Im9wZW5pZCIsImV4cCI6MTYwMzI2NTc5OCwiaWF0IjoxNjAzMjY1NDk4LCJjbGllbnRfb3Jnbm8iOiI5NzQ3NjEwNzYiLCJqdGkiOiJUUHNQZmNIeUV4UHVxOXpZV29TMW5WZ2VZYzdhcWZwVWZDaTA4Sm9pazlNIiwiY29uc3VtZXIiOnsiYXV0aG9yaXR5IjoiaXNvNjUyMy1hY3RvcmlkLXVwaXMiLCJJRCI6IjAxOTI6OTc0NzYxMDc2In19.ecF4tzmaaWW8ZEe5hbnDyQmxWo7bflCztQJINts_esT2XifPYJZ6BFBaM4INLGpbY_QTR4GIEhsIm_OZaRk_tFscKhJpqW5bgcAY0pBvpvb6hM1iyc708fD31_iGx

# Hent utkast og gjeldende
Her legger vi inn fødselsnummeret vi logget oss inn med, Dersom du velger et annet fødselsnummer så må den du logget på med ha tilgang til skattemeldingen du ønsker å hente

#### Parten nedenfor er brukt for internt test, pass på bruk deres egne testparter når dere tester

29114501318 har fått en myndighetsfastsetting

In [3]:
s = requests.Session()
s.headers = dict(idporten_header)
fnr="07116000738" #oppdater med test fødselsnummerene du har fått tildelt

### Utkast

In [4]:
url_utkast = f'https://mp-test.sits.no/api/eksterntapi/formueinntekt/skattemelding/utkast/2020/{fnr}'
r = s.get(url_utkast)
r

<Response [200]>

In [5]:
print(r.text)

<skattemeldingerOgNaeringsopplysningerforespoerselResponse xmlns="no:skatteetaten:fastsetting:formueinntekt:skattemeldingerognaeringsopplysninger:forespoersel:response:v1"><dokumenter><skattemeldingdokument><id>SKI:138:8243</id><encoding>utf-8</encoding><content>PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iVVRGLTgiPz48c2thdHRlbWVsZGluZyB4bWxucz0idXJuOm5vOnNrYXR0ZWV0YXRlbjpmYXN0c2V0dGluZzpmb3JtdWVpbm50ZWt0OnNrYXR0ZW1lbGRpbmc6ZWtzdGVybjp2OCI+PHBhcnRzcmVmZXJhbnNlPjMwMDAwMDY1NjQ8L3BhcnRzcmVmZXJhbnNlPjxpbm50ZWt0c2Fhcj4yMDIwPC9pbm50ZWt0c2Fhcj48YmFua0xhYW5PZ0ZvcnNpa3Jpbmc+PGtvbnRvPjxpZD4yYjQwZjgwMzZiMjhkMDBiNzVjNGQxODhkOGU5ZjNhOWY3MzQxYzNkPC9pZD48YmFua2Vuc05hdm4+PHRla3N0PlBBU1NFQkVLSyBPRyBST0FOIFJFVklTSk9OPC90ZWtzdD48L2JhbmtlbnNOYXZuPjxvcmdhbmlzYXNqb25zbnVtbWVyPjxvcmdhbmlzYXNqb25zbnVtbWVyPjkxMDk0MjAyNjwvb3JnYW5pc2Fzam9uc251bW1lcj48L29yZ2FuaXNhc2pvbnNudW1tZXI+PGtvbnRvbnVtbWVyPjx0ZWtzdD4yMTIzMjQyMzc1NzwvdGVrc3Q+PC9rb250b251bW1lcj48aW5uc2t1ZGQ+PGJlbG9lcD48YmVsb2VwSU5vaz48YmVsb2VwU29tSGVsdGFsbD45NTQwM

### Gjeldende

In [ ]:
url_gjeldende = f'https://mp-test.sits.no/api/eksterntapi/formueinntekt/skattemelding/2020/{fnr}'
r_gjeldende = s.get(url_gjeldende)
r_gjeldende

### Fastsatt
Her får en http 400 om vedkommende ikke har noen fastsetting

In [ ]:
url_fastsatt = f'https://mp-test.sits.no/api/eksterntapi/formueinntekt/skattemelding/fastsatt/2020/{fnr}'
r_fastsatt = s.get(url_fastsatt)
r_fastsatt
#r_fastsatt.elapsed.total_seconds()

## Svar fra hent gjeldende (utkast)
Legg merke til nytt element som viser en id for gjeldende skattemelding

In [ ]:
sjekk_svar = r_gjeldende

sme_og_naering_respons = xmltodict.parse(sjekk_svar.text)
skattemelding_base64 = sme_og_naering_respons["skattemeldingerOgNaeringsopplysningerforespoerselResponse"]["dokumenter"]["skattemeldingdokument"]
sme_utkast_base64 = skattemelding_base64["content"]
dokref = sme_og_naering_respons["skattemeldingerOgNaeringsopplysningerforespoerselResponse"]["dokumenter"]['skattemeldingdokument']['id']

decoded_xml = decode_dokument(skattemelding_base64)
sme_utkast = xml.dom.minidom.parseString(decoded_xml["content"]).toprettyxml()
print(xml.dom.minidom.parseString(sjekk_svar.text).toprettyxml())

#### I responsen er skattemeldingdokument Base64 encodet, kjør cellen nedenfor for å se det decodet

In [ ]:
print("*"*10+"\tBase64 decoded")
print(sme_utkast)

## Se skattemeldingen i SME 
https://skatt-ref.sits.no/web/skattemeldingen/2020

# Oppdater skattemeldingen og næringsmeldingen
### Oppdater Skattemeldingen
Som dere ser fra skattemeldingen, mangler kjøretøyet pris som ny. Nedenfor har dere xml elementet som mangler for kjøretøy som ny

In [ ]:
sme_dict = xmltodict.parse(sme_utkast)

pris_som_ny = xmltodict.parse("""<antattVerdiSomNyttKjoeretoey>
        <beloep>
          <beloepINok>
            <beloepSomHeltall>12345</beloepSomHeltall>
          </beloepINok>
          <beloepIValuta>
            <beloep>12345</beloep>
          </beloepIValuta>
          <valutakode>
            <valutakode>NOK</valutakode>
          </valutakode>
          <valutakurs>
            <valutakurs>1</valutakurs>
          </valutakurs>
        </beloep>
      </antattVerdiSomNyttKjoeretoey>""")
sme_dict["skattemelding"]["boligOgEiendeler"]["kjoeretoey"].update(pris_som_ny)

sme_xml = xmltodict.unparse(sme_dict)
sme_xml = sme_xml.replace('\t','').replace('\n','')
sme_xml_fix_base64 = base64.b64encode(sme_xml.encode("utf-8"))
sme_xml_fix_base64 = str(sme_xml_fix_base64.decode("utf-8"))

#Dette fungerer bare med myndighestfastsatt beløp på 1234567
sme_endret_myndighetsfastsetting = sme_xml.replace("1234567","7654321")
sme_endret_myndighetsfastsetting_b64 = base64.b64encode(sme_endret_myndighetsfastsetting.encode("utf-8"))
sme_endret_myndighetsfastsetting_b64 = str(sme_endret_myndighetsfastsetting_b64.decode("utf-8"))

In [ ]:
pris_som_ny = xmltodict.parse("""<antattVerdiSomNyttKjoeretoey>
        <beloep>
          <beloepINok>
            <beloepSomHeltall>12345</beloepSomHeltall>
          </beloepINok>
          <beloepIValuta>
            <beloep>12345</beloep>
          </beloepIValuta>
          <valutakode>
            <valutakode>NOK</valutakode>
          </valutakode>
          <valutakurs>
            <valutakurs>1</valutakurs>
          </valutakurs>
        </beloep>
      </antattVerdiSomNyttKjoeretoey>""")
if sme_dict["skattemelding"]["boligOgEiendeler"]["kjoeretoey"]:
    sme_dict["skattemelding"]["boligOgEiendeler"]["kjoeretoey"].update(pris_som_ny)

sme_xml = xmltodict.unparse(sme_dict)
sme_xml = sme_xml.replace('\t','').replace('\n','')
sme_xml_fix_base64 = base64.b64encode(sme_xml.encode("utf-8"))
sme_xml_fix_base64 = str(sme_xml_fix_base64.decode("utf-8"))

#Dette fungerer bare med myndighestfastsatt beløp på 1234567
sme_endret_myndighetsfastsetting = sme_xml.replace("1234567","7654321")
sme_endret_myndighetsfastsetting_b64 = base64.b64encode(sme_endret_myndighetsfastsetting.encode("utf-8"))
sme_endret_myndighetsfastsetting_b64 = str(sme_endret_myndighetsfastsetting_b64.decode("utf-8"))

## Næring xml
Her lager vi to nærings xml, en med en feil beløp i sumDriftsinntekt og en der dette er rettet

In [ ]:
with open("../eksempler/Naeringsingopplysninger-ENK-Eksempel2.xml", 'r') as f:
    naering_enk_xml = f.read()

naeringsopplysninger_enk_b64 = base64.b64encode(naering_enk_xml.encode("utf-8"))
naeringsopplysninger_enk_b64 = str(naeringsopplysninger_enk_b64.decode("utf-8"))


In [ ]:
valider_konvolutt = """
<skattemeldingerOgNaeringsopplysningerRequest xmlns="no:skatteetaten:fastsetting:formueinntekt:skattemeldingerognaeringsopplysninger:request:v1">
<dokumenter>
    <dokument>
        <type>skattemeldingPersonligSkattepliktig</type>
        <encoding>utf-8</encoding>
        <content>{skattemeldingPersonligSkattepliktig_base64}</content>
    </dokument>
    <dokument>
        <type>naeringsopplysninger</type>
        <encoding>utf-8</encoding>
        <content>{naeringsopplysninger_base64}</content>
    </dokument>
</dokumenter>
<dokumentreferanseTilGjeldendeDokument>
        <dokumenttype>skattemeldingPersonligSkattepliktig</dokumenttype>
        <dokumentidentifikator>{dok_ref}</dokumentidentifikator>
    </dokumentreferanseTilGjeldendeDokument>
</skattemeldingerOgNaeringsopplysningerRequest>
""".replace("\n","")


In [ ]:
naering_enk = valider_konvolutt.format(
    skattemeldingPersonligSkattepliktig_base64=sme_utkast_base64, #bruker utkastet uten noen endringer
    naeringsopplysninger_base64=naeringsopplysninger_enk_b64,
    dok_ref=dokref)

# Valider utkast sme med næringsopplysninger med feil sum

In [ ]:
def valider_sme(payload):
    url_valider = f'https://mp-test.sits.no/api/eksterntapi/formueinntekt/skattemelding/valider/2020/{fnr}'
    header = dict(idporten_header)
    header["content-type"] = "application/xml"
    return s.post(url_valider, headers=header, data=payload)
    


In [ ]:
valider_respons = valider_sme(naering_enk)
valider_respons

In [ ]:
#valider_respons.request.body

## Valideringsresultat av utkast og næringsoppgave

In [ ]:
print(xml.dom.minidom.parseString(valider_respons.text).toprettyxml())

## Vis valideringer i SME 
En kan åpne skattemeldingen og se de samme valideringsresultatene for skattemeldingen
https://skatt-ref.sits.no/web/skattemeldingen/2020

## Valider sme med fikset bil og sumpost i næring

In [ ]:
valider_respons_fix = valider_sme(valider_payload_fix)
valider_respons_fix

In [ ]:
valider_respons_fix.request.body

In [ ]:
#valider_respons_fix.text
ppxml = xml.dom.minidom.parseString(valider_respons_fix.text).toprettyxml()
start = ppxml.find("<avvikEtterBeregning>")
slutt = ppxml.find("</avvikEtterBeregning>")  + len("</avvikEtterBeregning>")
#print(ppxml[start:slutt])
print(ppxml)

# Altinn 3

## 1. Hent Altinn Token

In [ ]:
altinn_header = hent_altinn_token(idporten_header)
party_id = hent_party_id(altinn_header)

## 2. Opprett ny Instans i Altinn3

In [ ]:
instans_data = opprett_ny_instans(altinn_header, party_id)
instans_data

## Last opp metadata (skattemelding_V1)

In [ ]:
metadata_xml = """
<?xml version="1.0" encoding="utf-8"?>
<SkattemeldingApp xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema">
    <partsreferanse>str1234</partsreferanse>
    <inntektsaar>2020</inntektsaar>
    <innsendingstype>Komplett</innsendingstype>
    <opprettetAv>TurboSkatt AS..</opprettetAv>
    <opprettingstidspunkt>2012-06-03T12:12:12</opprettingstidspunkt>
    <endringstidspunkt>2012-06-03T12:12:12</endringstidspunkt>
</SkattemeldingApp>""".replace("\n","")

In [ ]:
req_metadata = last_opp_metadata(instans_data, altinn_header, xml=metadata_xml)
req_metadata

## Last opp skattemelding

In [ ]:
req_send_inn = last_opp_skattedata(instans_data, altinn_header, xml=valider_payload_fix)
req_send_inn

### Sett prosses til neste steg - Bekreftelse

In [ ]:
req_bekreftelse = endre_prosess_status(instans_data, altinn_header, "next")
req_bekreftelse

### Sett prosses til neste steg - Tilbakemelding

In [ ]:
req_bekreftelse = endre_prosess_status(instans_data, altinn_header, "next")
req_bekreftelse

### Se innsending i Altinn

Ta en slurk av kaffen og klapp deg selv på ryggen, du har nå sendt inn, la byråkratiet gjøre sin ting... og det tar litt tid

In [ ]:
instans_id=instans_data['id']
url = f'https://skd.apps.tt02.altinn.no/skd/sirius-skattemelding-v1#/instance/{instans_id}'
print(url)